# Preparing the data for model input

In [1]:
# DONE : Imports
from _Setup import *

In [2]:
train_data = pd.read_csv(rawTrainFile)
test_data = pd.read_csv(rawTestFile)

submission_data = pd.read_csv(rawSubmissionFile)

C:\Users\Trevo\AppData\Local\Temp\ipykernel_33812\3183785444.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv(rawTestFile)


In [3]:
def forward_fill_by_patient(df, patient_id_col='patient'):
    # Create a copy of the original DataFrame
    df_imputed = df.copy()

    df_imputed.loc[:, df_imputed.columns != patient_id_col] = (
        df_imputed.groupby(patient_id_col)
        .ffill()
    )
    
    # If there are patients with no entries for a column, those columns will remain unchanged
    return df_imputed

In [4]:
train_forward = forward_fill_by_patient(train_data)
train_forward.head()

,patient,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,...,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,16426,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,57.03,1.0,0.0,1.0,-4.25,1.0,0.0
1,16426,2,78.0,99.5,35.25,108.5,84.5,69.0,12.0,NaN,...,14.2,NaN,NaN,57.03,1.0,0.0,1.0,-4.25,2.0,0.0
2,16426,3,80.0,99.0,36.20,113.0,87.0,73.0,12.0,NaN,...,14.2,NaN,NaN,57.03,1.0,0.0,1.0,-4.25,3.0,0.0
3,16426,4,79.0,100.0,36.50,112.0,83.0,68.0,12.0,NaN,...,14.2,NaN,NaN,57.03,1.0,0.0,1.0,-4.25,4.0,0.0
4,16426,5,73.0,100.0,36.50,115.0,80.0,64.0,11.0,NaN,...,14.2,NaN,NaN,57.03,1.0,0.0,1.0,-4.25,5.0,0.0


In [5]:
def backward_fill_by_patient(df, patient_id_col='patient'):
    # Create a copy of the original DataFrame
    df_imputed = df.copy()
    
    df_imputed.loc[:, df_imputed.columns != patient_id_col] = (
        df_imputed.groupby(patient_id_col)
        .bfill()
    )
    
    # If there are patients with no entries for a column, those columns will remain unchanged
    return df_imputed

In [6]:
pre_imputed_train = backward_fill_by_patient(train_forward)
pre_imputed_train.head()

,patient,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,...,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,16426,1,78.0,99.5,35.25,108.5,84.5,69.0,12.0,NaN,...,14.2,NaN,162.0,57.03,1.0,0.0,1.0,-4.25,1.0,0.0
1,16426,2,78.0,99.5,35.25,108.5,84.5,69.0,12.0,NaN,...,14.2,NaN,162.0,57.03,1.0,0.0,1.0,-4.25,2.0,0.0
2,16426,3,80.0,99.0,36.20,113.0,87.0,73.0,12.0,NaN,...,14.2,NaN,162.0,57.03,1.0,0.0,1.0,-4.25,3.0,0.0
3,16426,4,79.0,100.0,36.50,112.0,83.0,68.0,12.0,NaN,...,14.2,NaN,162.0,57.03,1.0,0.0,1.0,-4.25,4.0,0.0
4,16426,5,73.0,100.0,36.50,115.0,80.0,64.0,11.0,NaN,...,14.2,NaN,162.0,57.03,1.0,0.0,1.0,-4.25,5.0,0.0


In [7]:
#Missing values after back-for imputation 
def missing_values_percentage(data, title):
    total_missing = data.isnull().sum()  
    percent_missing = (total_missing / len(data)) * 100  
    missing_data = pd.DataFrame({'Total Missing': total_missing, 'Percentage': percent_missing})
    missing_data = missing_data[missing_data['Total Missing'] > 0]  
    
   
    missing_data = missing_data.sort_values(by='Percentage', ascending=False)
    
    print(f"{title} - Missing Values:\n", missing_data)


missing_values_percentage(pre_imputed_train, 'Train Data')

Train Data - Missing Values:
                   Total Missing  Percentage
Bilirubin_direct        1012484   93.042344
EtCO2                    975057   89.602985
Fibrinogen               934300   85.857616
TroponinI                891709   81.943711
BaseExcess               693606   63.739010
Lactate                  689402   63.352683
SaO2                     683290   62.791020
Alkalinephos             650928   59.817110
Bilirubin_total          648138   59.560723
AST                      645965   59.361035
FiO2                     545205   50.101682
PaCO2                    528467   48.563541
HCO3                     528103   48.530092
pH                       513489   47.187136
PTT                      492396   45.248792
Chloride                 489811   45.011243
Unit2                    426031   39.150172
Unit1                    426031   39.150172
Phosphate                265449   24.393469
DBP                      181559   16.684387
Calcium                  104732    9.624360
Ma

## Note on backward_fill function:
The results of this code were run outside of this notebook, though imported through the tensor_decomp_pre_knn.pkl file.

# Using K Means Imputation

# Scale data
Data decomposed into train and test data using standardscaler, scaled, then recomposed

In [8]:
pre_imputed_train.head()

,patient,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,...,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,16426,1,78.0,99.5,35.25,108.5,84.5,69.0,12.0,NaN,...,14.2,NaN,162.0,57.03,1.0,0.0,1.0,-4.25,1.0,0.0
1,16426,2,78.0,99.5,35.25,108.5,84.5,69.0,12.0,NaN,...,14.2,NaN,162.0,57.03,1.0,0.0,1.0,-4.25,2.0,0.0
2,16426,3,80.0,99.0,36.20,113.0,87.0,73.0,12.0,NaN,...,14.2,NaN,162.0,57.03,1.0,0.0,1.0,-4.25,3.0,0.0
3,16426,4,79.0,100.0,36.50,112.0,83.0,68.0,12.0,NaN,...,14.2,NaN,162.0,57.03,1.0,0.0,1.0,-4.25,4.0,0.0
4,16426,5,73.0,100.0,36.50,115.0,80.0,64.0,11.0,NaN,...,14.2,NaN,162.0,57.03,1.0,0.0,1.0,-4.25,5.0,0.0


In [9]:
# forward fill 
train_data['HR'] = pd.to_numeric(train_data['HR'], errors='coerce')

#scaling while ommiting some variables 
indices = pre_imputed_train.isna()

input_to_imputer = pre_imputed_train.drop('SepsisLabel', axis = 1)

# Create Simple Imputer to fill NaN values with the mean
imputer = SimpleImputer(strategy='mean')
imputed_train = pd.DataFrame(imputer.fit_transform(input_to_imputer), columns=input_to_imputer.columns)

imputed_train['SepsisLabel'] = train_data['SepsisLabel']

# Identify columns to exclude from scaling
#exclude_cols = ['patient', 'time', 'Age', 'Gender', 'Unit1', 'Unit2', 'HospAdmTime', 'ICULOS']
exclude_cols = ['patient', 'time', 'Gender', 'Unit1', 'Unit2', 'HospAdmTime', 'ICULOS']

# Separate columns into those to scale and those to exclude
cols_to_scale = imputed_train.drop(columns=exclude_cols + ['SepsisLabel']).columns
scaled_data = imputed_train[cols_to_scale]

# Scale the selected columns
scaler = StandardScaler()
scaled_scaled_data = pd.DataFrame(scaler.fit_transform(scaled_data), columns=cols_to_scale)

# Combine scaled data with excluded columns and the target variable
scaled_train = pd.concat([scaled_scaled_data, imputed_train[exclude_cols], imputed_train['SepsisLabel']], axis=1)

# Display the resulting DataFrame
print(scaled_train.head())

         HR     O2Sat      Temp       SBP       MAP       DBP      Resp  \
0 -0.366247  0.747819 -2.225465 -0.654227  0.108104  0.372134 -1.285029   
1 -0.366247  0.747819 -2.225465 -0.654227  0.108104  0.372134 -1.285029   
2 -0.251474  0.588590 -0.911340 -0.460376  0.260250  0.680747 -1.285029   
3 -0.308860  0.907047 -0.496353 -0.503454  0.016817  0.294981 -1.285029   
4 -0.653182  0.907047 -0.496353 -0.374220 -0.165758 -0.013631 -1.478158   

          EtCO2  BaseExcess     HCO3  ...  Platelets       Age  patient  time  \
0  6.868005e-15    0.054560 -0.45495  ...   -0.44147 -0.299265  16426.0   1.0   
1  6.868005e-15    0.054560 -0.45495  ...   -0.44147 -0.299265  16426.0   2.0   
2  6.868005e-15    0.054560 -0.45495  ...   -0.44147 -0.299265  16426.0   3.0   
3  6.868005e-15   -0.931047 -0.45495  ...   -0.44147 -0.299265  16426.0   4.0   
4  6.868005e-15   -0.931047 -0.45495  ...   -0.44147 -0.299265  16426.0   5.0   

   Gender  Unit1  Unit2  HospAdmTime  ICULOS  SepsisLabel  
0 

In [10]:
scaled_train = scaled_train.dropna(subset=['SepsisLabel'])

In [11]:
# NOTE : Very Odd: 1 NAN for Sepsis Data occurring somewhere up here


# DONE : Use KMeans clusetering 
k = 8
k_means_class = KMeans(n_clusters= k, max_iter=300)

clusters = k_means_class.fit_predict(scaled_train.drop('SepsisLabel', axis = 1))

In [12]:
# DONE : Impute the missing values based upon means
for cluster in range(k):
    cluster_indices = np.where(clusters == cluster)[0]
    cluster_data = scaled_train.iloc[cluster_indices]
    cluster_mean = cluster_data.mean()

    for col in scaled_train.columns:
        for index in cluster_indices:
            if (indices.at[index, col] == True):
                scaled_train.at[index, col] = cluster_mean[col]

In [13]:
# DONE : Ensure the filtering worked
missing_values_percentage(scaled_train, 'Scaled Train Data')

Scaled Train Data - Missing Values:
                   Total Missing  Percentage
HR                            1    0.000092
Platelets                     1    0.000092
Magnesium                     1    0.000092
Phosphate                     1    0.000092
Potassium                     1    0.000092
Bilirubin_total               1    0.000092
Hct                           1    0.000092
Hgb                           1    0.000092
WBC                           1    0.000092
Age                           1    0.000092
O2Sat                         1    0.000092
patient                       1    0.000092
time                          1    0.000092
Gender                        1    0.000092
Unit1                         1    0.000092
Unit2                         1    0.000092
HospAdmTime                   1    0.000092
ICULOS                        1    0.000092
Lactate                       1    0.000092
Glucose                       1    0.000092
Bilirubin_direct              1    0.00

# Using Experimental Iterative Imputations

In [14]:
# DONE : Try the experimental imputer
# NOTE: THIS TAKES FOREVER, DON'T RUN UNLESS 100% NECESSARY
#imputer = IterativeImputer(random_state=0)
#train_data_imputed = imputer.fit(train_data)
# train_data_imputed = imputer.transform(train_data)
# train_data_imputed_df = pd.DataFrame(train_data_imputed, columns = imputer.get_feature_names_out())#, columns = imputer.get_feature_names_out())
# train_data_imputed_df.head(5)
# train_data_imputed_df.to_pickle("../Data/imputedData/iterative_imputed.pkl", index = False)

# SMOTE to increase proportion of Sepsis cases

In [16]:
scaled_train = scaled_train.dropna(subset=['SepsisLabel'])

X = scaled_train.drop(columns = ['SepsisLabel'])
y = scaled_train['SepsisLabel']


sm = SMOTE(sampling_strategy=0.3, k_neighbors=5, random_state=100)
X_train_syn, y_train_syn = sm.fit_resample(X, y)

print(scaled_train.iloc[:, 0:4].describe())
print(X_train_syn.iloc[:, 0:4].describe())

                 HR         O2Sat          Temp           SBP
count  1.088196e+06  1.088196e+06  1.088196e+06  1.088196e+06
mean   1.320076e-06  5.827757e-06  4.159469e-05  1.052050e-05
std    1.000000e+00  9.999760e-01  9.988701e-01  9.999883e-01
min   -3.694685e+00 -2.456949e+01 -4.960314e+01 -4.466632e+00
25%   -7.105684e-01 -3.667800e-01 -6.346818e-01 -7.188438e-01
50%   -7.931298e-02  2.701335e-01 -8.136589e-02 -1.157515e-01
75%    6.093293e-01  5.885903e-01  6.102790e-01  6.165749e-01
max    7.266204e+00  9.070471e-01  1.817806e+01  7.595214e+00
                 HR         O2Sat          Temp           SBP
count  1.389147e+06  1.389147e+06  1.389147e+06  1.389147e+06
mean   8.045142e-02 -1.287342e-02  8.232417e-02 -2.361252e-02
std    1.031378e+00  1.016276e+00  1.082561e+00  1.002043e+00
min   -3.694685e+00 -2.456949e+01 -4.960314e+01 -4.466632e+00
25%   -6.531815e-01 -3.667800e-01 -5.793502e-01 -7.619218e-01
50%    6.767295e-03  2.701335e-01  4.313019e-02 -1.157515e-01
75%    7

In [17]:
print(scaled_train['SepsisLabel'].describe())
print(y_train_syn.describe())

count    1.088196e+06
mean     1.803076e-02
std      1.330626e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: SepsisLabel, dtype: float64
count    1.389147e+06
mean     2.307690e-01
std      4.213250e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: SepsisLabel, dtype: float64


In [18]:
#merged data set 
resampled_train = pd.DataFrame(X_train_syn)
resampled_train['SepsisLabel'] = y_train_syn

In [19]:
# Before smote distribution 
sepsis_inf = scaled_train['SepsisLabel'].value_counts()
sepsis_percentages = (sepsis_inf / sepsis_inf.sum()) * 100
print(sepsis_percentages)

SepsisLabel
0.0    98.196924
1.0     1.803076
Name: count, dtype: float64


In [20]:
# After smote distribution
sepsis_inf2 = resampled_train['SepsisLabel'].value_counts()
sepsis_percentages2 = (sepsis_inf2 / sepsis_inf2.sum()) * 100
print(sepsis_percentages2)

SepsisLabel
0.0    76.923105
1.0    23.076895
Name: count, dtype: float64


In [21]:
# DONE : Convert to pickle
#resampled_train.to_pickle(kmeansTrainFilePickle)
#resampled_train.to_csv('imputed_train.csv', index=False)

 # Test Data preparation 


In [22]:
# forward fill 
test_data['HR'] = pd.to_numeric(test_data['HR'], errors='coerce')

test_filled = forward_fill_by_patient(test_data)
test_filled.head()

,patient,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,...,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS
0,6560,1,110.0,98.0,37.67,124.0,78.00,NaN,16.0,NaN,...,NaN,NaN,NaN,NaN,81.64,1.0,1.0,0.0,-0.03,2.0
1,6560,2,103.0,96.0,37.67,101.0,69.67,NaN,16.0,NaN,...,NaN,NaN,NaN,NaN,81.64,1.0,1.0,0.0,-0.03,3.0
2,6560,3,105.0,96.0,37.67,125.0,75.67,NaN,16.0,NaN,...,NaN,NaN,NaN,NaN,81.64,1.0,1.0,0.0,-0.03,4.0
3,6560,4,118.0,94.0,37.67,150.0,94.67,NaN,16.0,NaN,...,NaN,NaN,NaN,NaN,81.64,1.0,1.0,0.0,-0.03,5.0
4,6560,5,99.0,94.0,37.50,98.0,64.00,NaN,16.0,NaN,...,NaN,NaN,NaN,NaN,81.64,1.0,1.0,0.0,-0.03,6.0


In [23]:
# backward fill 
pre_imputed_test = backward_fill_by_patient(test_filled)
pre_imputed_test.head()

,patient,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,...,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS
0,6560,1,110.0,98.0,37.67,124.0,78.00,NaN,16.0,NaN,...,25.8,7.3,NaN,127.0,81.64,1.0,1.0,0.0,-0.03,2.0
1,6560,2,103.0,96.0,37.67,101.0,69.67,NaN,16.0,NaN,...,25.8,7.3,NaN,127.0,81.64,1.0,1.0,0.0,-0.03,3.0
2,6560,3,105.0,96.0,37.67,125.0,75.67,NaN,16.0,NaN,...,25.8,7.3,NaN,127.0,81.64,1.0,1.0,0.0,-0.03,4.0
3,6560,4,118.0,94.0,37.67,150.0,94.67,NaN,16.0,NaN,...,25.8,7.3,NaN,127.0,81.64,1.0,1.0,0.0,-0.03,5.0
4,6560,5,99.0,94.0,37.50,98.0,64.00,NaN,16.0,NaN,...,25.8,7.3,NaN,127.0,81.64,1.0,1.0,0.0,-0.03,6.0


In [24]:
# SCALING WHILE OMITING SOME VARIABLES FIXED# 
indices_test = pre_imputed_test.isna()

# Create Simple Imputer to fill NaN values with the mean for test data
# imputer = SimpleImputer(strategy='mean') -- needs to be the same imputer as used on the train data for correct scaling proportion
imputed_test = pd.DataFrame(imputer.transform(pre_imputed_test), columns=pre_imputed_test.columns)

# Separate columns into those to scale and those to exclude (no target variable in test)
cols_to_scale_test = imputed_test.drop(columns=exclude_cols).columns

# Scale the selected columns using the already-fitted scaler
scaled_test_data = pd.DataFrame(scaler.transform(imputed_test[cols_to_scale_test]), columns=cols_to_scale_test)

# Combine scaled data with excluded columns
scaled_test = pd.concat([scaled_test_data, imputed_test[exclude_cols]], axis=1)

In [25]:
indices_test.head()

,patient,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,...,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS
0,False,False,False,False,False,False,False,True,False,True,...,False,False,True,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,True,False,True,...,False,False,True,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,True,False,True,...,False,False,True,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,True,False,True,...,False,False,True,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,True,False,True,...,False,False,True,False,False,False,False,False,False,False


In [26]:

### FOR REFERENCE PURPOSES ONLY ###

# DONE : Impute the missing values based upon means
#for cluster in range(k):
#    cluster_indices = np.where(clusters == cluster)[0]
#    cluster_data = scaled_train.iloc[cluster_indices]
#    cluster_mean = cluster_data.mean()

#    for col in scaled_train.columns:
#        for index in cluster_indices:
#            if (indices.at[index, col] == True):
#                scaled_train.at[index, col] = cluster_mean[col]

In [27]:
clusters = k_means_class.predict(scaled_test)

# DONE : Impute the missing values based upon means
for cluster in range(k):
    cluster_indices = np.where(clusters == cluster)[0]
    cluster_data = scaled_train.iloc[cluster_indices]
    cluster_mean = cluster_data.mean()

    for col in scaled_test.columns:
        for index in cluster_indices:
            if (indices_test.at[index, col] == True):
                scaled_train.at[index, col] = cluster_mean[col]

In [28]:
# DONE : Ensure the filtering worked
# NOTE : These would all come out to 0 anyways with the mean imputation, no guarantee one the clustering with this check
print(scaled_test.isna().sum())

HR                  0
O2Sat               0
Temp                0
SBP                 0
MAP                 0
DBP                 0
Resp                0
EtCO2               0
BaseExcess          0
HCO3                0
FiO2                0
pH                  0
PaCO2               0
SaO2                0
AST                 0
BUN                 0
Alkalinephos        0
Calcium             0
Chloride            0
Creatinine          0
Bilirubin_direct    0
Glucose             0
Lactate             0
Magnesium           0
Phosphate           0
Potassium           0
Bilirubin_total     0
TroponinI           0
Hct                 0
Hgb                 0
PTT                 0
WBC                 0
Fibrinogen          0
Platelets           0
Age                 0
patient             0
time                0
Gender              0
Unit1               0
Unit2               0
HospAdmTime         0
ICULOS              0
dtype: int64


In [29]:
# DONE : Convert to pickle
# scaled_test.to_pickle(kmeansTestFilePickle)
#scaled_test.to_csv('scaled_test_data.csv', index=False)

 # Logistic Regression 

In [30]:
# simple logistic regression and predicting on test data 
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix,  f1_score

X_train = resampled_train.drop(columns=['SepsisLabel'])
y_train = resampled_train['SepsisLabel']

# Define features for the test set (assuming no target variable in test)
X_test = scaled_test.drop(columns=['SepsisLabel'], errors='ignore')  # Ignore error if 'SepsisLabel' is not in test

# Initialize the logistic regression model
logistic_model = LogisticRegression(random_state=100, max_iter = 500)

# Fit the model to the training data
logistic_model.fit(X_train, y_train)

y_pred_test = logistic_model.predict(X_test)

# If you have true labels for the test data, you can evaluate with confusion_matrix or classification_report
# Uncomment and replace y_true_test with actual test labels if available
# print("Confusion Matrix:\n", confusion_matrix(y_true_test, y_pred_test))
# print("\nClassification Report:\n", classification_report(y_true_test, y_pred_test))

# For demonstration, display predictions
print("Predictions on Test Data:\n", y_pred_test)

Predictions on Test Data:
 [0. 0. 0. ... 0. 0. 0.]


c:\Users\Trevo\anaconda3\envs\bigData\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [31]:
#replacing variable with preditions 
submission_data['SepsisLabel'] = y_pred_test

# Optionally, save the modified DataFrame to a CSV file
# TODO : create a variable for this path in _Setup.py
# submission_data.to_csv('submission_data_with_predictions.csv', index=False)

# Display the updated DataFrame (optional)
print(submission_data.head())

       ID  SepsisLabel
0  6560_1          0.0
1  6560_2          0.0
2  6560_3          0.0
3  6560_4          0.0
4  6560_5          0.0


In [32]:
#count number of rows 
num_rows = submission_data.shape[0]
print("Number of rows in submission_data:", num_rows)

Number of rows in submission_data: 464354


In [33]:
# logistic regression with fixed effects and predicting on trainning data 

X_train = resampled_train.drop(columns=['SepsisLabel', 'patient', 'time'], errors='ignore')
y_train = resampled_train['SepsisLabel']

# Step 2: Initialize the logistic regression model
logistic_model = LogisticRegression(random_state=100, max_iter=500)

# Fit the model to the training data
logistic_model.fit(X_train, y_train)

# Step 3: Make predictions on the training data
y_pred_train = logistic_model.predict(X_train)

# Step 4: Evaluate the model on the training data
#print("Training Predictions:\n", y_pred_train)
#print("Confusion Matrix (Training):\n", confusion_matrix(y_train, y_pred_train))
print("\nClassification Report (Training):\n", classification_report(y_train, y_pred_train))
f1 = f1_score(y_train, y_pred_train)
print("F1 Score (Training):", f1)


Classification Report (Training):
               precision    recall  f1-score   support

         0.0       0.81      0.97      0.88   1068575
         1.0       0.67      0.23      0.34    320572

    accuracy                           0.80   1389147
   macro avg       0.74      0.60      0.61   1389147
weighted avg       0.78      0.80      0.76   1389147

F1 Score (Training): 0.34288358121508167
